In [46]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import faiss
from transformers import AutoTokenizer, AutoModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
import json
from datetime import datetime

In [ ]:
DATASET_PATH = '../data/spam.csv'
df = pd.read_csv(DATASET_PATH)
messages = df['Message'].values.tolist()
labels = df['Category'].values.tolist()

In [48]:
MODEL_NAME = "intfloat/multilingual-e5-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
model.eval()
def average_pool(last_hidden_states, attention_mask):
    last_hidden = last_hidden_states.masked_fill(~attention_mask[...,None].bool(),0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[...,None]

In [ ]:
def get_embeddings(texts,model,tokenizer,device,batch_size=32):
    embeddings = []
    for i in tqdm(range(0,len(texts),batch_size), desc = "Generating embeddings"):
        batch_texts = texts[i:i+batch_size]
        batch_texts_with_prefix = [f"passage: {text}" for text in batch_texts]
        batch_dict = tokenizer(batch_texts_with_prefix, max_length = 512, padding = True, truncation = True)
        batch_dict = {k: torch.tensor(v).to(device) for k,v in batch_dict.items()}
        with torch.no_grad():
            outputs = model(**batch_dict)
            batch_embeddings = average_pool(outputs.last_hidden_state, batch_dict["attention_mask"])
            batch_embeddings = F.normalize(batch_embeddings, p=2, dim=1)
            embeddings.append(batch_embeddings.cpu().numpy())
    return np.vstack(embeddings)

le = LabelEncoder()
y = le.fit_transform(labels)
X_embeddings = get_embeddings(messages, model, tokenizer, device)
metadata = [{"index":i,"message":message, "label": label, "label_encoded":y[i]} 
            for i,(message, label) in enumerate(zip(messages,labels))]

Generating embeddings:   2%|█▎                                                         | 4/175 [00:34<24:03,  8.44s/it]

In [45]:
TEST_SIZE = 0.1
SEED = 42
train_indices, test_indices = train_test_split(range(len(messages)),test_size = TEST_SIZE, stratify = y, random_state = SEED)
X_train_emb = X_embeddings[train_indices]
X_test_emb = X_embeddings[test_indices]
y_train = y[train_indices]
y_test = y[test_indices]
train_metadata = [metadata[i] for i in train_indices]
test_metadata = [metadata[i] for i in test_indices]
embedding_dim = X_train_emb.shape[1]
index = faiss.IndexFlatIP(embedding_dim)
index.add(X_train_emb.astype("float32"))

In [39]:
def classify_with_knn(query_text,model,tokenizer,device,index,train_metadata,k=1):
    query_with_prefix = f"query: {query_text}"
    batch_dict = tokenizer([query_with_prefix],max_length=512,padding=True,truncation=True)
    batch_dict = {k:torch.tensor(v).to(device) for k,v in batch_dict.items()}
    with torch.no_grad():
        outputs = model(**batch_dict)
        query_embedding= average_pool(outputs.last_hidden_state, batch_dict["attention_mask"])
        query_embedding = F.normalize(query_embedding,p=2,dim=1)
        query_embedding = query_embedding.cpu().numpy().astype("float32")
    scores, indices = index.search(query_embedding,k)
    predictions= []
    neighbor_info = []
    for i in range(k):
        neighbor_idx = indices[0][i]
        neighbor_score = scores[0][i]
        neighbor_label = train_metadata[neighbor_idx]['label']
        neighbor_message = train_metadata[neighbor_idx]['message']
        predictions.append(neighbor_label)
        neighbor_info.append({
            "score": float(neighbor_score),
            "label": neighbor_label,
            "message":neighbor_message[:100] + "..." if len(neighbor_message) > 100 else neighbor_message
        })
    unique_labels, counts = np.unique(predictions, return_counts = True)
    final_prediction = unique_labels[np.argmax(counts)]
    return final_prediction, neighbor_info         

In [16]:
def evaluate_knn_accuracy(test_embeddings, test_labels, test_metadata, index, train_metadata, k_values=[1,3,5]):
    results = {}
    all_errors = {}
    for k in k_values:
        correct = 0
        total = len(test_embeddings)
        errors = []
        for i in tqdm(range(total),desc=f"Evaluating k={k}"):
            query_embedding = test_embeddings[i:i+1].astype("float32")
            true_label = test_metadata[i]["label"]
            true_message = test_metadata[i]["message"]
            scores, indices = index.search(query_embedding,k)
            predictions = []
            neighbor_details = []
            for j in range(k):
                neighbor_idx = indices[0][j]
                neighbor_label = train_metadata[neighbor_idx]["label"]
                neighbor_message = train_metadata[neighbor_idx]["message"]
                neighbor_score = float(scores[0][j])
                predictions.append(neighbor_label)
                neighbor_details.append({
                    "label": neighbor_label,
                    "message": neighbor_message,
                    "score":neighbor_score
                })
        unique_labels, counts = np.unique(predictions, return_counts=True)
        predicted_label = unique_labels[np.argmax(counts)]

        if predicted_label == true_label:
            correct += 1
        else:
            error_info = {
                "index" : i,
                "original_index" : test_metadata[i]["index"],
                "message" : true_message,
                "true_label" : true_label,
                "predicted_label" : predicted_label,
                "neighbors" : neighbor_details,
                "label_distribution" : {label: int(count) for label, count in zip(unique_labels,counts)}
            }
            errors.append(error_info)
        accuracy = correct / total
        error_count = total - correct
        results[k] = accuracy
        all_errors[k] = errors
        print(f"Accuracy with k = {k}: {accuracy:.4f}")
        print(f"Number of errors with k = {k}: {error_count}/{total} ({(error_count/total)*100:.2f}%)")
    return results, all_errors

In [42]:
%%time
print("Evaluating accuracy on test set...")
accuracy_results, error_results = evaluate_knn_accuracy(X_test_emb,y_test,test_metadata,index, train_metadata, k_values=[1,3,5])
print("\n" + "="*50)
print("ACCURACY RESULTS")
print("="*50)
for k, accuracy in accuracy_results.items():
    print(f"Top-{k} accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
print("="*50)

error_analysis = {
    "timestamp" : datetime.now().isoformat(),
    "model" : MODEL_NAME,
    "test_size": len(X_test_emb),
    "accuracy_results": accuracy_results,
    "errors_by_k" : {}
}
for k, errors in error_results.items():
    error_analysis["errors_by_k"][f"k_{k}"] = {
        "total_errors": len(errors),
        "error_rate" : len(errors) / len(X_test_emb),
        "errors" : errors
    }
output_file = "error_analysis.json"
with open(output_file,"w", encoding="utf-8") as f:
    json.dump(error_analysis, f, ensure_ascii=False, indent=2)
print(f"\n***Error analysis saved to: {output_file}***")
print()
print(f"***Summary:")
for k, errors in error_results.items():
    print(f"k = {k}: {len(errors)} errors out of {len(X_test_emb)} samples")

Evaluating accuracy on test set...


Evaluating k=1: 100%|███████████████████████████████████████████████████████████████| 558/558 [00:01<00:00, 317.75it/s]


Accuracy with k = 1: 0.0018
Number of errors with k = 1: 557/558 (99.82%)


Evaluating k=3: 100%|███████████████████████████████████████████████████████████████| 558/558 [00:01<00:00, 327.77it/s]


Accuracy with k = 3: 0.0018
Number of errors with k = 3: 557/558 (99.82%)


Evaluating k=5: 100%|███████████████████████████████████████████████████████████████| 558/558 [00:01<00:00, 321.50it/s]

Accuracy with k = 5: 0.0018
Number of errors with k = 5: 557/558 (99.82%)

ACCURACY RESULTS
Top-1 accuracy: 0.0018 (0.18%)
Top-3 accuracy: 0.0018 (0.18%)
Top-5 accuracy: 0.0018 (0.18%)

***Error analysis saved to: error_analysis.json***

***Summary:
k = 1: 0 errors out of 558 samples
k = 3: 0 errors out of 558 samples
k = 5: 0 errors out of 558 samples
CPU times: total: 4.97 s
Wall time: 5.23 s


In [43]:
def spam_classifier_pipeline(user_input, k = 3):
    print()
    print(f"***Classifying: '{user_input}'")
    print()
    print(f"***Using top-{k} nearest neighbors")
    print()

    prediction,neighbors = classify_with_knn(user_input, model, tokenizer, device, index, train_metadata,k=k)
    print(f"*** Prediction: {prediction.upper()}")
    print()

    print("***Top neighbors:")
    for i, neighbor in enumerate(neighbors,1):
        print(f"{i}. Label {neighbor["label"]} | Score: {neighbor['score']:.4f}")
        print(f"Message: {neighbor["message"]}")
        print()
    labels = [n["label"] for n in neighbors]
    label_counts = {label: labels.count(label) for label in set(labels)}
    return {
        "prediction": prediction,
        "neighbors" : neighbors,
        "label_distribution": label_counts
    }

In [44]:
test_examples = ["I am actually thinking a way of doing something useful",
                "FREE!! Click here to win $1000 NOW! Limited time offer"]
for i, example in enumerate(test_examples,1):
    print(f"\n--- Exxample {i}: '{example}' ---")
    resul = spam_classifier_pipeline(example, k = 3)
print("\n--- Interactive Tessting ---")
user_text = "Win a free iPhone! Click here now!"
k_value = 5
result = spam_classifier_pipeline(user_text, k = k_value)


--- Exxample 1: 'I am actually thinking a way of doing something useful' ---

***Classifying: 'I am actually thinking a way of doing something useful'

***Using top-3 nearest neighbors

*** Prediction: HAM

***Top neighbors:
1. Label ham | Score: 0.8424
Message: yeah, that's what I was thinking

2. Label ham | Score: 0.8412
Message: that would be good … I'll phone you tomo lunchtime, shall I, to organise something?

3. Label ham | Score: 0.8344
Message: See? I thought it all through


--- Exxample 2: 'FREE!! Click here to win $1000 NOW! Limited time offer' ---

***Classifying: 'FREE!! Click here to win $1000 NOW! Limited time offer'

***Using top-3 nearest neighbors

*** Prediction: SPAM

***Top neighbors:
1. Label spam | Score: 0.8575
Message: FREE MESSAGE Activate your 500 FREE Text Messages by replying to this message with the word FREE For...

2. Label spam | Score: 0.8575
Message: FREE MESSAGE Activate your 500 FREE Text Messages by replying to this message with the word FREE For